In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
original_df = pd.read_csv('apartments_for_rent_classified_10K_utf.csv')

In [3]:
df = original_df[["amenities", "bathrooms", "bedrooms", "fee", "price", "price_type", "square_feet", "cityname", "state", "time"]]

In [4]:
df.head(5)

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
0,NaN,NaN,0.0,No,790,Monthly,101,Washington,DC,1577359415
1,NaN,NaN,1.0,No,425,Monthly,106,Evansville,IN,1577017063
2,NaN,1.0,0.0,No,1390,Monthly,107,Arlington,VA,1577359410
3,NaN,1.0,0.0,No,925,Monthly,116,Seattle,WA,1576667743
4,NaN,NaN,0.0,No,880,Monthly,125,Arlington,VA,1577359401


In [5]:
df.isna().mean()

amenities      0.3549
bathrooms      0.0034
bedrooms       0.0007
fee            0.0000
price          0.0000
price_type     0.0000
square_feet    0.0000
cityname       0.0077
state          0.0077
time           0.0000
dtype: float64

## Convert timestamp to datetime objects

In [6]:
df['time'] = pd.to_datetime(df['time'], unit='s')

In [7]:
df.head(2)

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
0,NaN,NaN,0.0,No,790,Monthly,101,Washington,DC,2019-12-26 11:23:35
1,NaN,NaN,1.0,No,425,Monthly,106,Evansville,IN,2019-12-22 12:17:43


## Normalizing Price and Square Feet


Convert any prices measured in Weeks to Months by converting to price per day then price per year then price per month (roughly 4.348 multiplier).

There is one abnormally high price with 'weekly' price_type. The price for 'week' is 1560, and digging into the dataset, we found that it is a duplicate of index 15. The listing is for the same place except it's on another listing website. This duplicate will be dropped in favor of the other one.

There is also one record with 'Monthly|Weekely' price_type with a price of 275.0, and the price is likely to be on a weekly basis.

In [8]:
df[df['price'] == 1560][:2]

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
15,"AC,Basketball,Cable or Satellite,Gym,Internet ...",1.0,1.0,No,1560,Weekly,200,New Bern,NC,2019-12-17 21:27:56
16,"AC,Basketball,Cable or Satellite,Gym,Internet ...",1.0,1.0,No,1560,Monthly,200,New Bern,NC,2019-12-15 10:37:53


In [9]:
df = df.drop([16,17])

In [10]:
df[df['price_type'] == 'Monthly|Weekly']

,amenities,bathrooms,bedrooms,fee,price,price_type,square_feet,cityname,state,time
235,"Cable or Satellite,Pool,Refrigerator,Storage,TV",NaN,NaN,No,275,Monthly|Weekly,300,Lakeland,FL,2019-11-27 21:43:20


In [11]:
for i in df.index:
  if (df['price_type'][i] == "Weekly") or (df['price_type'][i] == "Monthly/Weekly"):
    df.loc[i, 'price'] = df['price'][i] / 7 * 365.25 / 12

df['price'] = (df['price'] - df['price'].min()) / (df['price'].max() - df['price'].min())

In [12]:
print(df['price'].min())
print(df['price'].max())

0.0
1.0


Since we have standardized the units of the price, we can drop the price_type column.

In [13]:
df = df.drop(columns=['price_type'])

## Normalize the square_feet using Min Max method

In [14]:
df['square_feet'] = (df['square_feet'] - df['square_feet'].min()) / (df['square_feet'].max() - df['square_feet'].min())

In [15]:
print(df['square_feet'].min())
print(df['square_feet'].max())

0.0
1.0


## Impute NaN values using median

In [16]:
df['bathrooms'] = df['bathrooms'].replace({np.nan: df['bathrooms'].median()})

In [17]:
df['bedrooms'] = df['bedrooms'].replace({np.nan: df['bedrooms'].median()})

## Preprocessing on the amenities

In [18]:
df['amenities'] = np.array(df['amenities'].str.strip().str.split(","))

In [19]:
df

,amenities,bathrooms,bedrooms,fee,price,square_feet,cityname,state,time
0,NaN,1.0,0.0,No,0.011281,0.000000,Washington,DC,2019-12-26 11:23:35
1,NaN,1.0,1.0,No,0.004302,0.000125,Evansville,IN,2019-12-22 12:17:43
2,NaN,1.0,0.0,No,0.022753,0.000150,Arlington,VA,2019-12-26 11:23:30
3,NaN,1.0,0.0,No,0.013862,0.000376,Seattle,WA,2019-12-18 11:15:43
4,NaN,1.0,0.0,No,0.013002,0.000602,Arlington,VA,2019-12-26 11:23:21
...,...,...,...,...,...,...,...,...,...
9995,NaN,4.0,5.0,No,0.110899,0.155367,Edina,MN,2019-11-30 11:22:55
9996,NaN,8.0,6.0,No,0.474187,0.215920,Montecito,CA,2019-12-26 11:40:19
9997,NaN,8.5,6.0,No,0.206501,0.281135,Potomac,MD,2019-12-26 11:42:40
9998,"[Basketball, Cable or Satellite, Doorman, Hot ...",1.0,1.0,No,0.087763,1.000000,New York,NY,2019-12-26 12:09:46


In [20]:
df['amenities'] = df['amenities'].replace({np.nan: 'None'})

In [21]:
df.head(5)

,amenities,bathrooms,bedrooms,fee,price,square_feet,cityname,state,time
0,None,1.0,0.0,No,0.011281,0.000000,Washington,DC,2019-12-26 11:23:35
1,None,1.0,1.0,No,0.004302,0.000125,Evansville,IN,2019-12-22 12:17:43
2,None,1.0,0.0,No,0.022753,0.000150,Arlington,VA,2019-12-26 11:23:30
3,None,1.0,0.0,No,0.013862,0.000376,Seattle,WA,2019-12-18 11:15:43
4,None,1.0,0.0,No,0.013002,0.000602,Arlington,VA,2019-12-26 11:23:21


## Assign a unique value to each state and city
Unique values has been assigned to cities and states, np.nan will have a value of 0.

In [22]:
all_states = df['state'].unique()
all_states

array(['DC', 'IN', 'VA', 'WA', 'NY', 'CA', 'AZ', 'NC', 'GA', 'FL', nan,
       'AL', 'MD', 'CO', 'NM', 'IL', 'TN', 'AK', 'MA', 'NJ', 'OR', 'DE',
       'PA', 'TX', 'IA', 'SC', 'MN', 'MI', 'KY', 'WI', 'OH', 'CT', 'RI',
       'NV', 'UT', 'MO', 'OK', 'NH', 'NE', 'LA', 'ND', 'AR', 'KS', 'ID',
       'HI', 'MT', 'VT', 'SD', 'WV', 'MS', 'ME', 'WY'], dtype=object)

In [23]:
rep_vals = {}
for i in range(len(all_states)):
    rep_vals[all_states[i]] = i

In [24]:
rep_vals[np.nan] = 0
rep_vals['DC'] = 10

In [25]:
df['state_num'] = df['state'].replace(rep_vals)

In [26]:
df.head(5)

,amenities,bathrooms,bedrooms,fee,price,square_feet,cityname,state,time,state_num
0,None,1.0,0.0,No,0.011281,0.000000,Washington,DC,2019-12-26 11:23:35,10
1,None,1.0,1.0,No,0.004302,0.000125,Evansville,IN,2019-12-22 12:17:43,1
2,None,1.0,0.0,No,0.022753,0.000150,Arlington,VA,2019-12-26 11:23:30,2
3,None,1.0,0.0,No,0.013862,0.000376,Seattle,WA,2019-12-18 11:15:43,3
4,None,1.0,0.0,No,0.013002,0.000602,Arlington,VA,2019-12-26 11:23:21,2


In [27]:
all_cities = df['cityname'].unique()

In [28]:
rep_city_vals = {}
for i in range(len(all_cities)):
    rep_city_vals[all_cities[i]] = i

In [29]:
rep_city_vals[np.nan] = 0
rep_city_vals['Washington'] = 13

In [30]:
df['city_num'] = df['cityname'].replace(rep_city_vals)

In [31]:
df.head(5)

,amenities,bathrooms,bedrooms,fee,price,square_feet,cityname,state,time,state_num,city_num
0,None,1.0,0.0,No,0.011281,0.000000,Washington,DC,2019-12-26 11:23:35,10,13
1,None,1.0,1.0,No,0.004302,0.000125,Evansville,IN,2019-12-22 12:17:43,1,1
2,None,1.0,0.0,No,0.022753,0.000150,Arlington,VA,2019-12-26 11:23:30,2,2
3,None,1.0,0.0,No,0.013862,0.000376,Seattle,WA,2019-12-18 11:15:43,3,3
4,None,1.0,0.0,No,0.013002,0.000602,Arlington,VA,2019-12-26 11:23:21,2,2


In [32]:
df['fee'].unique()

array(['No'], dtype=object)

In [33]:
final_cleaned = df.drop(columns=['fee'])
final_cleaned

,amenities,bathrooms,bedrooms,price,square_feet,cityname,state,time,state_num,city_num
0,None,1.0,0.0,0.011281,0.000000,Washington,DC,2019-12-26 11:23:35,10,13
1,None,1.0,1.0,0.004302,0.000125,Evansville,IN,2019-12-22 12:17:43,1,1
2,None,1.0,0.0,0.022753,0.000150,Arlington,VA,2019-12-26 11:23:30,2,2
3,None,1.0,0.0,0.013862,0.000376,Seattle,WA,2019-12-18 11:15:43,3,3
4,None,1.0,0.0,0.013002,0.000602,Arlington,VA,2019-12-26 11:23:21,2,2
...,...,...,...,...,...,...,...,...,...,...
9995,None,4.0,5.0,0.110899,0.155367,Edina,MN,2019-11-30 11:22:55,26,628
9996,None,8.0,6.0,0.474187,0.215920,Montecito,CA,2019-12-26 11:40:19,5,1534
9997,None,8.5,6.0,0.206501,0.281135,Potomac,MD,2019-12-26 11:42:40,12,80
9998,"[Basketball, Cable or Satellite, Doorman, Hot ...",1.0,1.0,0.087763,1.000000,New York,NY,2019-12-26 12:09:46,4,752
